# SMS Spam Detection

In [88]:
import pandas as pd

# Read in data from csv
data = pd.read_csv("spam.csv", encoding='latin-1')

# Drop columns with NaN values
for col in data.columns:
    if 'Unnamed' in col:
        data.drop(columns=col, inplace=True)

# Rename columns to useful names
data = data.rename(columns={"v1":"Label", "v2":"Text"})

# Add an encoding for the labels: ham = 0, spam = 1
enc_labels = []
for item in data["Label"]:
    enc_labels.append(0 if item == "ham" else 1)
data.insert(loc=2, column="encoded_label", value=enc_labels)
print(data.head())


  Label                                               Text  encoded_label
0   ham  Go until jurong point, crazy.. Available only ...              0
1   ham                      Ok lar... Joking wif u oni...              0
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...              1
3   ham  U dun say so early hor... U c already then say...              0
4   ham  Nah I don't think he goes to usf, he lives aro...              0


In [87]:
# Find the average number of tokens in all sentences
sentences = data["Text"].values
print(sentences)

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'
 'Ok lar... Joking wif u oni...'
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"
 ... 'Pity, * was in mood for that. So...any other suggestions?'
 "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free"
 'Rofl. Its true to its name']
